In [2]:
%matplotlib inline

In [31]:
from bentso.constants import ENTSO_COUNTRIES, TRADE_PAIRS
from collections import defaultdict
from correspondentia import load_csv
from pathlib import Path
import bentso
import brightway2 as bw
import numpy as np
import os
import pandas as pd
import presamples
import seaborn as sb

from wurst.ecoinvent.electricity_markets import (
    move_all_generation_to_high_voltage, 
    remove_electricity_trade,
    empty_high_voltage_markets,
    all_providers,
)
import wurst

In [4]:
assert bentso.__version__ >= (0, 2, 2)

In [5]:
bw.projects.set_current("SETAC 2019")

In [6]:
YEAR = 2017

# Create presample packages

They have two components - the indices, and the data itself. The indices list in the input and output activities. The data has rows of exchanges to be substituted, and columns of values to choose from.

## Preparing the bentso output

Need to do the following:

1. Remove `Other` and `Other renewable` (functionality added to `bentso` 0.2)
2. Split entso labels to ecoinvent labels, using the prepared correspondence tables and `correspondentia`
3. Link abstract ecoinvent labels to specific activity keys, using backups when necessary
4. Write the presample packages into a campaign

In [7]:
NEW_DB_NAME = "ecoinvent 3.5 ENTSO-E"

In [8]:
kwh_dict = {(ds['name'], ds['location']): ds['code'] 
            for ds in bw.Database("ecoinvent 3.5 cutoff")
            if ds['unit'] == 'kilowatt hour'}

In [9]:
base_dir = Path(os.getcwd()).resolve() / "corr_tables"

In [10]:
c = bentso.CachingDataClient(key="cache-only")

Using data directory /Users/cmutel/Library/Application Support/bentso


In [ ]:
ecoinvent = wurst.extract_brightway2_databases(["ecoinvent 3.5 cutoff"])

In [10]:
ecoinvent = move_all_generation_to_high_voltage(ecoinvent)
ecoinvent = remove_electricity_trade(ecoinvent)
ecoinvent = empty_high_voltage_markets(ecoinvent)

16022
16022
16022
16022


In [11]:
def format_exchange(db_name, code, amount):
    return {
        'amount': amount,
        'type': 'technosphere',
        'input': (db_name, code),
    }

In [11]:
import_from = defaultdict(set)

for k, v in TRADE_PAIRS.items():
    # Switch from exporter: [importers] to importer: [exporters]
    for country in v:
        import_from[country].add(k)

In [13]:
for country in ENTSO_COUNTRIES:
    ct = load_csv(base_dir / country / "corr_table.csv")
    
    df = c.get_generation(country, YEAR, True).sum()
    trade = [(exporter, c.get_trade(exporter, country, YEAR).sum()) 
             for exporter in import_from[country]]
    total = df.sum() + sum(x[1] for x in trade)
    
    market = next(x for x in ecoinvent 
                  if x['name'] == 'market for electricity, high voltage' 
                  and x['location'] == country)
    
    for label, amount in zip(df.index, df):
        for mapping in ct[label]:
            name = mapping['value']
            market['exchanges'].append(format_exchange(
                NEW_DB_NAME, 
                kwh_dict.get((name, country), kwh_dict.get((name, 'DE'))),
                amount / total * mapping['weight']))
            
    for key, value in trade:
        market['exchanges'].append(format_exchange(
            "ecoinvent 3.5 ENTSO-E", 
            kwh_dict[('market for electricity, high voltage', key)],
            value / total))

In [14]:
wurst.write_brightway2_database(ecoinvent, NEW_DB_NAME)

16022 datasets
543113 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:31


Title: Writing activities to SQLite3 database:
  Started: 05/14/2019 11:49:31
  Finished: 05/14/2019 11:50:03
  Total time elapsed: 00:00:31
  CPU %: 86.00
  Memory %: 10.39
Created database: ecoinvent 3.5 ENTSO-E


## Create presample packages

In [70]:
indices, matrix = [], []

for country in ENTSO_COUNTRIES:
    ct = load_csv(base_dir / country / "corr_table.csv")
    
    df = c.get_generation(country, YEAR, clean=True, full_year=True)
    if df.shape[0] != 8760:
        print(country, df.shape)

    trade = [(exporter, c.get_trade(exporter, country, YEAR, full_year=True)) 
             for exporter in import_from[country]]
    total = df.sum(axis=1) + sum(x[1] for x in trade)
    
    market = (NEW_DB_NAME, kwh_dict[('market for electricity, high voltage', country)])
    
    for label in df:
        for mapping in ct[label]:
            name = mapping['value']
            code = kwh_dict.get((name, country), kwh_dict.get((name, 'DE')))
            
            indices.append(((NEW_DB_NAME, code), market, 'technosphere'))
            matrix.append((df[label] / total).values * mapping['weight'])
            
    for key, value in trade:
        indices.append((
            (NEW_DB_NAME, kwh_dict[('market for electricity, high voltage', key)]), 
            market, 
            'technosphere'
        ))
        matrix.append((value / total).values)

In [71]:
matrix = np.vstack([o.reshape((1, -1)) for o in matrix])

In [72]:
mask = np.isnan(matrix)
matrix[mask] = 0

In [73]:
package = presamples.create_presamples_package(
    matrix_data=[(matrix, indices, 'technosphere')], 
    name='ENTSO-E 2017'
)

In [41]:
campaign = presamples.Campaign.get_or_create(name="ENTSO-E 2017")

In [29]:
campaign.add_local_presamples(package[1], copy=False)